# Segmenting and Clustering the neighborhood in Toronto, answer part 1

### 1. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [93]:
import pandas as pd 

In [94]:
!conda install -c conda-forge lxml --yes #install lxml library for processing XML and HTML in the Python language

Solving environment: done

# All requested packages already installed.



### 2.The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [208]:
df_pc = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]#more than one table in the page, we choose the first one that we need
df_pc.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### 3.Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [209]:
df_pc = df_pc[df_pc['Borough']!='Not assigned']
df_pc.reset_index(drop = True, inplace  = True)
df_pc.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### 4.More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [210]:
temp = df_pc.groupby("Postal Code")['Neighbourhood'].apply(lambda Neighbourhood: ', '.join(Neighbourhood))
temp = pd.DataFrame(temp)
temp = pd.merge(temp, df_pc,how='left', on=['Postal Code'])
temp.head()

,Postal Code,Neighbourhood_x,Borough,Neighbourhood_y
0,M1B,"Malvern, Rouge",Scarborough,"Malvern, Rouge"
1,M1C,"Rouge Hill, Port Union, Highland Creek",Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Woburn,Scarborough,Woburn
4,M1H,Cedarbrae,Scarborough,Cedarbrae


In [211]:
df_pc = temp[['Postal Code', 'Borough', 'Neighbourhood_x']]
df_pc = df_pc.rename(columns = {'Neighbourhood_x':'Neighborhood'}).reset_index(drop=True)
df_pc.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [212]:
#check if the postal code is unique
if (df_pc['Postal Code'].unique().size == df_pc.shape[0]):
    print('Unique postal code in dataframe column \'Postal Code\'')

Unique postal code in dataframe column 'Postal Code'


### 5. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [214]:
#check if there is any item column 'Neighborhood' is 'Not assigned'
for index,value in enumerate(df_pc['Neighborhood']):
    if (value == 'Not assigned'):
        df_pc.loc[index,'Neighborhood']=df_pc.loc[index,'Borough']
        print('there is column that in column \'Neighborhood\' not assigned, the value is renamed as Borough')
df_pc.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [215]:
df_pc.shape

(103, 3)